In [9]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from eq_odds_postprocessing import EqOddsPostprocessing
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso
from random import sample

## Huangrui's Dataset 

In [10]:
default_mappings = {
    #Huangrui flip the lable
    'label_maps': [{0: 'Did recid.', 1: 'No recid.'}],
    'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'},
                                 {1.0: 'Caucasian', 0.0: 'Not Caucasian'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class CompasDataset(StandardDataset):
    """ProPublica COMPAS Dataset.

    See :file:`aif360/data/raw/compas/README.md`.
    """

    def __init__(self, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[],
                 features_to_drop=[], na_values=[],
                 custom_preprocessing=None,
                 metadata=default_mappings,
                 path='./Huangrui/recidivism/recidivism_test1.csv'):

    
        df = pd.read_csv(path,index_col=False)
        df.rename(columns={'Probationerssex_Female': 'sex'}, inplace=True)
        df.drop(["Probationerssex_Male","Probationerssex_Notascertained"], axis=1, inplace=True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        #flip the Y lable to 0: recid, 1: no recid
        df["Y"] = [1 if x == 0 else 0 for x in df["Y"]]
        
        super(CompasDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


#### Load dataset and specify options

In [11]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [12]:
experiments_info = {}
budget = 1
for K in range(1, 6):
    # dataset_orig_train= Zindi(path="./Huangrui/zindi/zindi_train{}.csv".format(K),protected_attribute_names=['sex'],
    #             privileged_classes=[[1]])
    dataset_orig_test= CompasDataset(path="./Huangrui/recidivism/recidivism_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    # #only use the budget% of the training data
    # dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)
    # Lasso linear classifier and predictions
    lmod = pickle.load(open('experiments/recidivism'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    data= pickle.load(open("./experiments/recidivism{}_sex.pkl".format(K), "rb"))
    X_test = data["Xtest"]
    X_test["Probationerssex_Female"] = [0]*X_test.shape[0]
    y_test = dataset_orig_test_pred.labels
    y_test_pred = lmod.predict(X_test)
    #flip the lable
    dataset_orig_test_pred.labels = (y_test_pred<0.5).reshape(-1,1)
    metric_test = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                                  unprivileged_groups, privileged_groups)
   
    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels), "Equal opportunity difference": metric_test["Equal opportunity difference"]}

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6859
Statistical parity difference = -0.0837
Disparate impact = 0.8973
Average odds difference = -0.0453
Equal opportunity difference = -0.0002
Theil index = 0.1217
K = 1, budget = 1
The Error for the test dataset is 0.2687
The Equal opportunity difference for the test dataset is -0.0001884


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.7173
Statistical parity difference = -0.1293
Disparate impact = 0.8412
Average odds difference = -0.0646
Equal opportunity difference = -0.0721
Theil index = 0.1275
K = 2, budget = 1
The Error for the test dataset is 0.2477
The Equal opportunity difference for the test dataset is -0.07209


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6851
Statistical parity difference = -0.0480
Disparate impact = 0.9069
Average odds difference = 0.0012
Equal opportunity difference = 0.0168
Theil index = 0.3034
K = 3, budget = 1
The Error for the test dataset is 0.3302
The Equal opportunity difference for the test dataset is 0.01683


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.7069
Statistical parity difference = -0.0663
Disparate impact = 0.9091
Average odds difference = -0.0426
Equal opportunity difference = 0.0376
Theil index = 0.1544
K = 4, budget = 1
The Error for the test dataset is 0.2649
The Equal opportunity difference for the test dataset is 0.03763
Balanced accuracy = 0.7308
Statistical parity difference = -0.0780
Disparate impact = 0.8969
Average odds difference = -0.0367
Equal opportunity difference = -0.0320
Theil index = 0.1293
K = 5, budget = 1
The Error for the test dataset is 0.2384
The Equal opportunity difference for the test dataset is -0.03203


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [13]:
experiments_info

{'K = 1, budget = 1': {'Error': 0.2686695278969957,
  'Equal opportunity difference': -0.000188422277463407},
 'K = 2, budget = 1': {'Error': 0.24774581365392873,
  'Equal opportunity difference': -0.07209356908152087},
 'K = 3, budget = 1': {'Error': 0.3301846285959639,
  'Equal opportunity difference': 0.01683322537861265},
 'K = 4, budget = 1': {'Error': 0.2649205667668527,
  'Equal opportunity difference': 0.03762792298357154},
 'K = 5, budget = 1': {'Error': 0.23840206185567012,
  'Equal opportunity difference': -0.03203099510603591}}